In [200]:
import pandas as pd
import json
import pyodbc as Context

In [3]:
Path_File = "C:\csv-data-collect/github-2023_06_31_contributors_top_20_stars.csv"

In [197]:
def dataSpliter(Path, JSON_Percentage , SQL_Percentage ):
    # vars
    #CSV_Percentage = 0
    # read the csv 
    df = pd.read_csv(Path)
    Toatal_Percentage = JSON_Percentage + SQL_Percentage

    if ( Toatal_Percentage == 0):
        return df
    if (Toatal_Percentage <= 100):

        #CSV_Percentage  = (JSON_Percentage - SQL_Percentage) if (JSON_Percentage > SQL_Percentage)  else (SQL_Percentage - JSON_Percentage)
        df = df.sample(frac=1,random_state= 1).reset_index()

        Size_0_JSON = round((len(df) * JSON_Percentage) / 100)

        Size_0_SQL = round((len(df) * SQL_Percentage) / 100)

        #Size_0_CSV = len(df) - (Size_0_JSON + Size_0_SQL)

        df_JSON = df[:Size_0_JSON]

        df_SQL = df[Size_0_JSON : Size_0_JSON + Size_0_SQL]

        df_CSV = df[Size_0_JSON + Size_0_SQL : ]

        #print(Size_0_JSON,Size_0_SQL,Size_0_CSV)
        return df_JSON , df_SQL, df_CSV
    else:
        raise Exception("The Percentage should be Equal or less then 100")

In [199]:
try:
    df_JSON, df_SQL, df_CSV = dataSpliter(Path_File, 2, 1)
    df_JSON.to_json(path_or_buf='C:/Users/Youcode/desktop/data_Splitter/export.json',orient='records')
except Exception as e:
    print(e.args[0])

In [231]:
# Insert Into Sql database
server = 'LAPTOP-VQCV01FE\SQLEXPRESS' 
database = 'data_splitter' 
username = '' 
password = '' 
Connection = Context.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password,autocommit=True)
cursor = Connection.cursor()

cursor.execute("SELECT @@version;") 
row = cursor.fetchone() 
while row: 
    print(row[0])
    row = cursor.fetchone()

Microsoft SQL Server 2022 (RTM) - 16.0.1000.6 (X64) 
	Oct  8 2022 05:58:25 
	Copyright (C) 2022 Microsoft Corporation
	Express Edition (64-bit) on Windows 10 Pro 10.0 <X64> (Build 22621: ) (Hypervisor)



In [230]:
for row in df_SQL.itertuples():
    cursor.execute(
        f'''
            INSERT INTO Commits
            VALUES (?,?,?)
        ''',
        row.full_name,
        row.contributor,
        row.commit
    )